# Sioux Falls example

In [1]:
import sys
sys.path.append('../..')

## File paths

In [2]:
fldr = 'D:/release/Sample models/sioux_falls_2020_02_15'

# remove the comments for the lines below to run the Chicago model example instead
# fldr = 'D:/release/Sample models/Chicago_2020_02_15'
# proj_name = 'chicagomodel.sqlite'

dt_fldr = '0_tntp_data'
prj_fldr = '1_project'
skm_fldr = '2_skim_results'
assg_fldr = '4_assignment_results'
dstr_fldr = '5_distribution_results'
frcst_fldr = '6_forecast'
ftr_fldr = '7_future_year_assignment'

## Some logging

## Opening the project

In [3]:
# Imports
from aequilibrae.project import Project
from os.path import join

In [4]:
project = Project()
project.load(join(fldr, prj_fldr))

## Path computation

In [5]:
# imports
from aequilibrae.paths import PathResults, path_computation

In [6]:
# we build all graphs
project.network.build_graphs()
# We get warnings that several fields in the project are filled with NaNs.  Which is true, but we won't use those fields

../..\aequilibrae\project\network\network.py:257: UserWarning: Fields were removed from Graph for being non-numeric: modes,link_type
  warn(f'Fields were removed from Graph for being non-numeric: {",".join(removed_fields)}')
../..\aequilibrae\paths\graph.py:360: UserWarning: Field name has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')
../..\aequilibrae\paths\graph.py:360: UserWarning: Field lanes has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')


In [7]:
# we grab the graph for cars
graph = project.network.graphs['c']

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['free_flow_time', 'distance'])

# And we will allow paths to be compute going through other centroids/centroid connectors
# required for the Sioux Falls network, as all nodes are centroids
graph.set_blocked_centroid_flows(False)

# instantiate a path results object and prepare it to work with the graph
res = PathResults()
res.prepare(graph)

# compute a path from node 2 to 13
path_computation(2, 13, graph, res)

In [8]:
# We can get the sequence of nodes we traverse
res.path_nodes

array([ 2,  1,  3, 12, 13], dtype=int64)

In [9]:
# We can get the link sequence we traverse
res.path

array([ 3,  2,  7, 37], dtype=int64)

In [10]:
# We can get the mileposts for our sequence of nodes
res.milepost

array([ 0.,  6., 10., 14., 17.])

In [11]:
# If we want to compute the path for a different destination and same origin, we can just do this
# It is way faster when you have large networks
res.update_trace(4)

In [12]:
res.path_nodes

array([2, 6, 5, 4], dtype=int64)

## Skimming

In [13]:
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
from aequilibrae.paths import NetworkSkimming, SkimResults

In [14]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_blocked_centroid_flows(False)

In [15]:
# setup the object result
res = SkimResults()
res.prepare(graph)

In [16]:
# And run the skimming
skm = NetworkSkimming(graph, res)
skm.execute()

In [17]:
# The result is an AequilibraEMatrix object
skims = res.skims

# We can export to AEM and OMX
skims.export(join(fldr, skm_fldr, 'skimming_on_time.aem'))
skims.export(join(fldr, skm_fldr, 'skimming_on_time.omx'))

In [18]:
# We can access each matrix
skims.free_flow_time

memmap([[ 0.,  6.,  4.,  8., 10., 11., 16., 13., 15., 18., 14.,  8., 11.,
         18., 23., 18., 20., 18., 22., 22., 18., 20., 17., 15.],
        [ 6.,  0., 10., 11.,  9.,  5., 10.,  7., 14., 16., 17., 14., 17.,
         21., 19., 12., 14., 12., 16., 16., 22., 21., 23., 21.],
        [ 4., 10.,  0.,  4.,  6., 10., 15., 12., 11., 14., 10.,  4.,  7.,
         14., 19., 17., 19., 17., 21., 20., 14., 16., 13., 11.],
        [ 8., 11.,  4.,  0.,  2.,  6., 11.,  8.,  7., 10.,  6.,  8., 11.,
         10., 15., 13., 15., 13., 17., 17., 18., 18., 14., 15.],
        [10.,  9.,  6.,  2.,  0.,  4.,  9.,  6.,  5.,  8.,  8., 10., 13.,
         12., 14., 11., 13., 11., 15., 15., 19., 17., 16., 17.],
        [11.,  5., 10.,  6.,  4.,  0.,  5.,  2.,  9., 11., 12., 14., 17.,
         16., 14.,  7.,  9.,  7., 11., 11., 17., 16., 20., 20.],
        [16., 10., 15., 11.,  9.,  5.,  0.,  3., 12.,  9., 14., 19., 19.,
         17., 12.,  5.,  7.,  2.,  9.,  6., 12., 11., 15., 15.],
        [13.,  7., 12.,  8.

# Traffic assignment with skimming

In [19]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [20]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

In [21]:
import sys
# Because assignment takes a long time, we want the log to be shown here
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [22]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view(['matrix']) # We will only assign one user class stored as 'matrix' inside the OMX file

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)


# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

2020-11-05 13:24:31,582;aequilibrae;INFO ; bfw Assignment STATS
2020-11-05 13:24:31,582;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-11-05 13:24:31,707;aequilibrae;INFO ; 1,inf,1.0
2020-11-05 13:24:31,819;aequilibrae;INFO ; 2,0.8485503509703024,0.36497345609427145
2020-11-05 13:24:31,937;aequilibrae;INFO ; 3,0.3813926225800203,0.2298356924660528
2020-11-05 13:24:32,064;aequilibrae;INFO ; 4,0.19621277462606984,0.18591312145268074
2020-11-05 13:24:32,188;aequilibrae;INFO ; 5,0.09069073200924213,0.7090816523174254
2020-11-05 13:24:32,308;aequilibrae;INFO ; 6,0.20600048221061426,0.1229016022154401
2020-11-05 13:24:32,438;aequilibrae;INFO ; 7,0.06710568925282254,0.38638656717489844
2020-11-05 13:24:32,567;aequilibrae;INFO ; 8,0.10307514154369488,0.1093055036410267
2020-11-05 13:24:32,713;aequilibrae;INFO ; 9,0.04222147191362779,0.2487805192125393
2020-11-05 13:24:32,851;aequilibrae;INFO ; 10,0.05926435464772421,0.15904810628271004
2020-11-05 13:24:32,982;aequilibrae;INFO ; 11,0.

### Outputs


In [23]:
# Convergence report is easy to see
import pandas as pd
convergence_report = assig.report()
convergence_report.head()

,iteration,rgap,alpha,warnings,beta0,beta1,beta2
0,1,inf,1.000000,,1.000000,0.000000,0.000000
1,2,0.848550,0.364973,,1.000000,0.000000,0.000000
2,3,0.381393,0.229836,,1.000000,0.000000,0.000000
3,4,0.196213,0.185913,,0.959771,0.040229,0.000000
4,5,0.090691,0.709082,,0.687640,0.286705,0.025654


In [24]:
volumes = assig.results()
volumes.head()

,matrix_ab,matrix_ba,matrix_tot,Congested_Time_AB,Congested_Time_BA,Congested_Time_Max,Delay_factor_AB,Delay_factor_BA,Delay_factor_Max,VOC_AB,VOC_BA,VOC_max,PCE_AB,PCE_BA,PCE_tot
link_id,,,,,,,,,,,,,,,
1,4570.421761,NaN,4570.421761,6.0,NaN,6.0,1.0,NaN,1.0,0.176463,NaN,0.176463,4570.421761,NaN,4570.421761
2,8275.382482,NaN,8275.382482,4.0,NaN,4.0,1.0,NaN,1.0,0.353596,NaN,0.353596,8275.382482,NaN,8275.382482
3,4675.373252,NaN,4675.373252,6.0,NaN,6.0,1.0,NaN,1.0,0.180515,NaN,0.180515,4675.373252,NaN,4675.373252
4,5900.513362,NaN,5900.513362,5.0,NaN,5.0,1.0,NaN,1.0,1.190056,NaN,1.190056,5900.513362,NaN,5900.513362
5,8170.430991,NaN,8170.430991,4.0,NaN,4.0,1.0,NaN,1.0,0.349112,NaN,0.349112,8170.430991,NaN,8170.430991


In [25]:
# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c.csv'), output="loads")
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c.aed'), output="loads")

In [26]:
# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,assg_fldr, 'skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,assg_fldr, 'skims.omx'))

    

# Trip distribution

### Calibration

We will calibrate synthetic gravity models using the skims for TIME that we just generated

In [27]:
import numpy as np
from aequilibrae.distribution import GravityCalibration
from aequilibrae.matrix import AequilibraeMatrix

In [28]:
# We need the demand
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))

# And the skims
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))

In [29]:
# But before using the data, let's get some impedance for the intrazonals
# Let's assume it is 75% of the closest zone

# If we run the code below more than once, we will be overwriting the diagonal values with non-sensical data
# so let's zero it first
np.fill_diagonal(imped.matrix['time_final'], 0)

# We compute it with a little bit of NumPy magic
intrazonals = np.amin(imped.matrix['time_final'], where=imped.matrix['time_final']>0, initial=imped.matrix['time_final'].max(), axis=1)
intrazonals *= 0.75

# Then we fill in the impedance matrix
np.fill_diagonal(imped.matrix['time_final'], intrazonals)


In [30]:
# We set the matrices forbeing used in computation
imped.computational_view(['time_final'])
demand.computational_view(['matrix'])

In [31]:
from math import log10, floor
def plot_tlfd(demand, skim, name):
    import matplotlib.pyplot as plt
    b = floor(log10(skim.shape[0]) * 10)
    n, bins, patches = plt.hist(np.nan_to_num(skim.flatten(),0), bins = b, weights=np.nan_to_num(demand.flatten()), density=False, facecolor='g', alpha=0.75)

    plt.xlabel('Trip length')
    plt.ylabel('Probability')
    plt.title('Trip-length frequency distribution')
    plt.savefig(name, format="png")
    plt.clf()

In [32]:
for function in ['power', 'expo']:
    model = GravityCalibration(matrix=demand, impedance=imped, function=function, nan_as_zero=True)
    model.calibrate()
    
    # we save the model
    model.model.save(join(fldr, dstr_fldr, f'{function}_model.mod'))
    
    # We save a trip length frequency distribution image
    plot_tlfd(model.result_matrix.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, f'{function}_tfld.png') )
    
    # We can save the result of applying the model as well
    # we can also save the calibration report
    with open(join(fldr, dstr_fldr, f'{function}_convergence.log'), 'w') as otp:
        for r in  model.report:
            otp.write(r+'\n')

<Figure size 432x288 with 0 Axes>

In [33]:
# We save a trip length frequency distribution image
plot_tlfd(demand.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, 'demand_tfld.png') )

<Figure size 432x288 with 0 Axes>

# Forecast

* We create a set of *'future'* vectors using some random growth factors
* We apply the model for inverse power, as the TFLD seems to be a better fit for the actual one

In [34]:
from aequilibrae.distribution import Ipf, GravityApplication, SyntheticGravityModel, Ipf
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
import numpy as np

In [35]:
# We compute the vectors from our matrix
mat = AequilibraeMatrix()

mat.load(join(fldr, dt_fldr, 'demand.omx'))
mat.computational_view()
origins = np.sum(mat.matrix_view, axis=1)
destinations = np.sum(mat.matrix_view, axis=0)

args = {'file_path':join(fldr,  frcst_fldr, 'synthetic_future_vector.aed'),
        "entries": mat.zones, 
        "field_names": ["origins", "destinations"],
    "data_types": [np.float64, np.float64], 
        "memory_mode": False}

vectors = AequilibraeData()
vectors.create_empty(**args)

vectors.index[:] =mat.index[:]

# Then grow them with some random growth between 0 and 10% - Plus balance them
vectors.origins[:] = origins * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations[:] = destinations * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations *= vectors.origins.sum()/vectors.destinations.sum()

In [36]:
# Impedance 
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))
imped.computational_view(['time_final'])

# We want the main diagonal to be zero
np.fill_diagonal(imped.matrix_view, np.nan)

In [37]:
for function in ['power', 'expo']:
    model = SyntheticGravityModel()
    model.load(join(fldr, dstr_fldr, f'{function}_model.mod'))

    outmatrix = join(fldr,frcst_fldr, f'demand_{function}_model.aem') 
    apply = GravityApplication()
    args = {"impedance": imped,
            "rows": vectors,
            "row_field": "origins",
            "model": model,
            "columns": vectors,
            "column_field": "destinations",
            "output": outmatrix,
            "nan_as_zero":True
            }

    gravity = GravityApplication(**args)
    gravity.apply()

    #We get the output matrix and save it to OMX too
    resm = AequilibraeMatrix()
    resm.load(outmatrix)
    resm.export(join(fldr,frcst_fldr, f'demand_{function}_model.omx'))

### We now run IPF for the future vectors

In [38]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view()

args = {'matrix': demand,
        'rows': vectors,
        'columns': vectors,
        'column_field': "destinations",
        'row_field': "origins",
        'nan_as_zero': True}

ipf = Ipf(**args)
ipf.fit()

output = AequilibraeMatrix()
output.load(ipf.output.file_path)

output.export(join(fldr,frcst_fldr, 'demand_ipf.aem'))
output.export(join(fldr,frcst_fldr, 'demand_ipf.omx'))


# Future traffic assignment

In [39]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [40]:
logger.info('\n\n\n TRAFFIC ASSIGNMENT FOR FUTURE YEAR')

2020-11-05 13:24:39,246;aequilibrae;INFO ; 


 TRAFFIC ASSIGNMENT FOR FUTURE YEAR


In [41]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

In [42]:
# Let's use the IPF matrix
demand = AequilibraeMatrix()
demand.load(join(fldr, frcst_fldr, 'demand_ipf.omx'))
demand.computational_view() # There is only one matrix there, so don;t even worry about its core name

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

2020-11-05 13:24:39,677;aequilibrae;INFO ; bfw Assignment STATS
2020-11-05 13:24:39,677;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-11-05 13:24:39,790;aequilibrae;INFO ; 1,inf,1.0
2020-11-05 13:24:39,910;aequilibrae;INFO ; 2,0.8802711249340913,0.3383782509024761
2020-11-05 13:24:40,034;aequilibrae;INFO ; 3,0.480289682785832,0.2227580391025405
2020-11-05 13:24:40,155;aequilibrae;INFO ; 4,0.2518851824049047,0.17297427780976923
2020-11-05 13:24:40,281;aequilibrae;INFO ; 5,0.12360171808243732,0.4900586029613198
2020-11-05 13:24:40,416;aequilibrae;INFO ; 6,0.18679595660091808,0.140147469362504
2020-11-05 13:24:40,539;aequilibrae;INFO ; 7,0.09086142722249475,0.35770324266429077
2020-11-05 13:24:40,670;aequilibrae;INFO ; 8,0.10497094366609536,0.8480798469896671
2020-11-05 13:24:40,815;aequilibrae;INFO ; 9,0.1371010902267951,0.14152271182437456
2020-11-05 13:24:40,947;aequilibrae;INFO ; 10,0.06860667962998532,0.16343971341248112
2020-11-05 13:24:41,083;aequilibrae;INFO ; 11,0.0364

2020-11-05 13:24:51,594;aequilibrae;INFO ; 94,0.00010236968709981051,0.017458477307306253
2020-11-05 13:24:51,713;aequilibrae;INFO ; 95,9.825221202534195e-05,0.016737138294739766
2020-11-05 13:24:51,849;aequilibrae;INFO ; 96,8.553885077103601e-05,0.008695327593620888
2020-11-05 13:24:51,991;aequilibrae;INFO ; 97,8.769449795741627e-05,0.010925525647240697
2020-11-05 13:24:52,129;aequilibrae;INFO ; 98,9.871569875441917e-05,0.008015220188558164
2020-11-05 13:24:52,246;aequilibrae;INFO ; 99,7.822592439841203e-05,0.006772101579543307
2020-11-05 13:24:52,382;aequilibrae;INFO ; 100,8.274145859998298e-05,0.0070637960387709125
2020-11-05 13:24:52,526;aequilibrae;INFO ; 101,7.790842946105245e-05,0.0065180147902365645
2020-11-05 13:24:52,654;aequilibrae;INFO ; 102,7.947846838072463e-05,0.01329953430005701
2020-11-05 13:24:52,787;aequilibrae;INFO ; 103,7.509578969852205e-05,0.013193345301932297
2020-11-05 13:24:52,911;aequilibrae;INFO ; 104,7.953264404553255e-05,0.010376585109088034
2020-11-05 13:

2020-11-05 13:25:03,332;aequilibrae;INFO ; 186,3.1818721055404206e-05,0.0046773675541063485
2020-11-05 13:25:03,487;aequilibrae;INFO ; 187,4.01648831743481e-05,0.006935403054193095
2020-11-05 13:25:03,627;aequilibrae;INFO ; 188,2.7034503526302145e-05,0.0020971790852228555
2020-11-05 13:25:03,777;aequilibrae;INFO ; 189,2.2435319105336875e-05,0.004209650079521295
2020-11-05 13:25:03,901;aequilibrae;INFO ; 190,2.811729766924176e-05,0.0033926048797328127
2020-11-05 13:25:04,034;aequilibrae;INFO ; 191,2.6353456568977493e-05,0.00441516159365563
2020-11-05 13:25:04,162;aequilibrae;INFO ; 192,2.060410240027124e-05,0.0027365523598458924
2020-11-05 13:25:04,279;aequilibrae;INFO ; 193,1.9156927955395665e-05,0.0026805936007122874
2020-11-05 13:25:04,399;aequilibrae;INFO ; 194,2.0548465470603766e-05,0.0030207870863191108
2020-11-05 13:25:04,517;aequilibrae;INFO ; 195,2.5239283434470908e-05,0.0028544757426833323
2020-11-05 13:25:04,635;aequilibrae;INFO ; 196,1.9860048822402067e-05,0.0026107433588293

2020-11-05 13:25:14,914;aequilibrae;INFO ; 277,2.9966875872385603e-05,0.021002210019704655
2020-11-05 13:25:15,043;aequilibrae;INFO ; 278,3.24499043975059e-05,0.018909127931611078
2020-11-05 13:25:15,173;aequilibrae;INFO ; 279,2.8050845749102548e-05,0.017710297185284513
2020-11-05 13:25:15,298;aequilibrae;INFO ; 280,2.4196857249724538e-05,0.0188922172321996
2020-11-05 13:25:15,426;aequilibrae;INFO ; 281,2.307059317034514e-05,0.01564388823299164
2020-11-05 13:25:15,546;aequilibrae;INFO ; 282,2.2653998539414784e-05,0.011255192001970726
2020-11-05 13:25:15,672;aequilibrae;INFO ; 283,1.7973505705490174e-05,0.01252550489226385
2020-11-05 13:25:15,806;aequilibrae;INFO ; 284,2.1714898982816563e-05,0.0069181137671650425
2020-11-05 13:25:15,926;aequilibrae;INFO ; 285,1.8952015353933952e-05,0.010148948817191134
2020-11-05 13:25:16,050;aequilibrae;INFO ; 286,2.2654157539284674e-05,0.010247931539508384
2020-11-05 13:25:16,174;aequilibrae;INFO ; 287,1.7310364969216597e-05,0.005539647000988024
2020-

In [43]:

# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, ftr_fldr, 'future_link_flows_c.csv'), output="loads")
assigclass.results.save_to_disk(join(fldr, ftr_fldr, 'future_link_flows_c.aed'), output="loads")

# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,ftr_fldr, 'future_skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,ftr_fldr, 'future_skims.omx'))
